In [7]:
from supabase import create_client,Client
import os
# get environment variables from .env file
import os

def load_env():
    with open('.env', 'r') as f:
        for line in f:
            if line.strip():
                key, value = line.strip().split('=', 1)
                os.environ[key] = value

load_env()


# get environment variables
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)



In [105]:
# functions to load data
import json
import requests

def get_translations(count):
    url = 'https://read.84000-translate.org/translations.json?api-version=0.4.0'
    response = requests.get(url)
    works = json.loads(response.text)
    # only return the required number of translations
    return works['work'][:count]

def get_work(Work_id):
    link = f"../../data-json/{Work_id}.json"
    print("loading file",link)
    # fet the json data from the local file
    with open(link, 'r') as file:
        work = json.loads(file.read())
    return work

def make_title(work,WorkId):
    titles = work['work'][0]['title']
    titles_dicts = [dict({'work_uuid': WorkId,
                          'type': title['titleType'],
                          'content': title['content'],
                          'language': title['language'],
                          'migrationId': title['titleMigrationId']}) for title in titles]
    return titles_dicts

def make_passages(work,WorkId):
    passages = work['translation']['passage']
    passage_dict = [dict({'work_uuid': WorkId,
                          'xmlId': passage['xmlId'],
                          'parent': passage['parentId'],
                          'label': passage['passageLabel'],
                          'type': passage['segmentationType'],
                          'sort': passage['passageSort'],
                          'content': passage['content']}) for passage in passages]
    return passage_dict

def make_work(Current_Work,translation):
    title = translation['work'][0]['title'][0]['content']
    work_dict = dict({'xmlId': Current_Work['workId'],
                      'url': Current_Work['url'],
                      'type': Current_Work['workType'],
                      'toh': Current_Work['catalogueWorkIds'],
                      'title': title,
                      'migrationJson': translation})
    return work_dict

In [107]:
# upsert into supabase
def store_work(work_dict):
    # check if the work already exists with the same xmlId
    # if exists return the workId
    # else insert the work and return the workId
    does = None
    does = supabase.table('works').select('uuid').eq('xmlId',work_dict['xmlId']).execute()
    if (does.count != None):
        print("Work Already Exists")
        return does.data[0]['uuid']
    else :
        try:
            work_data,count = supabase.table('works').insert(work_dict).execute()
            return work_data[1][0]['uuid']
        except Exception as e:
            print ("Error Occured",e)
            return None
    
    
def store_titles(titles_dicts):
    try:
        title_data,count = supabase.table('titles').upsert(titles_dicts).execute()
        print("Added",len(title_data[1]))
        return "success"
    except Exception as e:
        print("Error Occured",e)
        return e
        # If error occurs delete all just inserted data
        # try:
        #     if (title_data[1]):
        #         for inserts in title_data[1]:
        #             print(inserts)
        #             data_temp,count_temp = supabase.table('titles').delete().eq('id',inserts['id']).execute()
        #             print('deleted',data_temp[1])
        # except IndexError:
        #     print("No Data to Delete")
            
def store_passages(passage_dict):
    try:
        passage_data,count = supabase.table('passages').upsert(passage_dict).execute()
        print("Added",len(passage_data[1]))
        return "success"
    except Exception as e:
        print("Error Occured",e)
        return e
        # If error occurs delete all just inserted data
        # try:
        #     if (passage_data[1]):
        #         for inserts in passage_data[1]:
        #             print(inserts)
        #             data_temp,count_temp = supabase.table('passages').delete().eq('id',inserts['id']).execute()
        #             print('deleted',data_temp[1])
        # except IndexError:
        #     print("No Data to Delete")


In [103]:
# Excuter 
# 1. get required  number of translations
# 2. Load the data into Supabase and get back the workId
# 3. Get the work data from the local file
# 4. Get the titles and passages
# 5. Create Title and passage dictionary with WorkId

def load_data(translation_count):
    works = get_translations(translation_count)
    for work in works:
        work_uuid = None
        try:
            WorkId = work['workId']
            print("Processing current:",WorkId)
            translation = get_work(WorkId)
            work_dict = make_work(work,translation)
            print("Storing Work")
            work_uuid = store_work(work_dict)
            if (work_uuid != None):
                print("Work UUID",work_uuid)
                print("Loading Titles")
                titles_dicts = make_title(translation,work_uuid)
                print("Loading Passages")
                passage_dict = make_passages(translation,work_uuid)
                print("Storing Title")
                return_titles = store_titles(titles_dicts)
                if (return_titles != "success"):
                    raise Exception(return_titles)
                print("Storing Passages")
                return_passages = store_passages(passage_dict)
                if (return_passages != "success"):
                    raise Exception(return_passages)
                
        except Exception as e:
            print("Error Occured",e)
            # delete work cascade
            try:
                print('error in',WorkId)
                # store error as string
                supabase.table('Error').upsert({'error':str(e),'xmlId':WorkId}).execute()
                if (work_uuid):
                    data_temp,count_temp = supabase.table('works').delete().eq('uuid',work_uuid).execute()
                    
                    print('deleted',data_temp[1])
            except IndexError:
                print("No Data to Delete")
            
    
    return


In [109]:
load_data(10)

Processing current: UT22084-001-001
loading file ../../data-json/UT22084-001-001.json
Storing Work
Work UUID 028e739d-2871-4266-9ffa-b34776208a5d
Loading Titles
Loading Passages
Storing Title and Passages
Added 11
Added 1421
Processing current: UT22084-001-002
loading file ../../data-json/UT22084-001-002.json
Storing Work
Work UUID 5c77fcf5-c32b-4d6b-81fc-f8ba6e5e5752
Loading Titles
Error Occured 'translation'
error in UT22084-001-002
deleted [{'uuid': '5c77fcf5-c32b-4d6b-81fc-f8ba6e5e5752', 'type': 'eft:translation', 'toh': 'toh1-2', 'xmlId': 'UT22084-001-002', 'title': 'Chapter 2: On the Rite of Restoration', 'migrationJson': {'work': [{'url': '/translation/UT22084-001-002.json?api-version=0.4.0&annotate=false', 'title': [{'content': 'Chapter 2: On the Rite of Restoration', 'language': 'en', 'titleType': 'eft:mainTitle', 'titleMigrationId': 'toh1-2/eft:mainTitle/en/1'}, {'content': 'Poṣadhavastu', 'language': 'Sa-Ltn', 'titleType': 'eft:mainTitle', 'titleMigrationId': 'toh1-2/eft:mai